<a href="https://colab.research.google.com/github/Teera235/The-6th-Kibo-Robot-Programming-Challenge/blob/main/Train_Kibo_YoloV8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#กำหนด Path ที่จะทำงานใน Colab

In [6]:
import os
HOME = os.getcwd()
print(HOME)

/content/{HOME}/datasets


#ติดตั้ง Library ที่จำเป็น

In [ ]:
!pip install ultralytics==8.2.103 -q
!pip install roboflow -q
!pip uninstall -y numpy
!pip install numpy==1.24.4


# สร้าง Folder รองรับ Dataset

In [ ]:
!mkdir -p {HOME}/datasets
%cd {HOME}/datasets

#โหลด Dataset จาก Roboflow

In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="ijp1CpxrIPTwscQJxiOz")
project = rf.workspace("tee-dhqjm").project("kibo_object-detection-2025")
version = project.version(4)
dataset = version.download("yolov8")


#  Train Model Yolo ด้วย Dataset ที่เตรียมไว้

In [ ]:
import ultralytics
from ultralytics import YOLO
%cd {HOME}
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=500 imgsz=800 plots=True

## API d642e7a710956cbd046dd1914760b5f4b02e0033

#แสดงผลลัพธ์จากการ Train Model

## เตรียมฟังก์ชั่น แสดงรูปภาพ

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

def display_image(image_path, figsize=(10, 10)):
    """แสดงภาพโดยใช้ matplotlib"""
    if not os.path.exists(image_path):
        print(f"ไม่พบไฟล์ที่ {image_path}")
        return
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.axis('off')
    plt.show()

##แสดง Confusion Metrix

In [ ]:
image_path = f"{HOME}/runs/detect/train/confusion_matrix.png"
display_image(image_path)

##แสดงภาพตัวอย่างที่ทำนายแล้ว จากการ Valid

In [ ]:
image_path = f'{HOME}/runs/detect/train/val_batch0_pred.jpg'
display_image(image_path)

#ดาวน์โหลด Model ไปใช้

In [ ]:
from google.colab import files
file_path = '/content/runs/detect/train/weights/best.pt'
files.download(file_path)

# การนำโมเดลที่สำเร็จแล้วมาใช้ (ใน Colab)

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
from google.colab import files
import os

## อัพโหลดไฟล์รูปที่ต้องการค้นหาวัตถุ

In [ ]:

uploaded = files.upload()
if not uploaded:
    raise FileNotFoundError("No file uploaded!")
test_image_path = next(iter(uploaded.keys()))

## เรียกใช้โมเดลที่ Train ไว้แล้ว

In [ ]:
model_path = "/content/best.pt"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"YOLO model file not found at {model_path}")
model = YOLO(model_path)

## อ่านภาพที่อัพโหลดผ่าน Model

In [ ]:
image = cv2.imread(test_image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at {test_image_path}")

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = model(image)

## ตีกรอบ (Bounding Box) วัตถุจากผลลัพธ์ของ Model

In [ ]:
for result in results:
    boxes = result.boxes.cpu().numpy()
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        confidence = box.conf[0]
        cls = int(box.cls[0])
        if confidence > 0.5:
            label = f"{model.names[cls]} {confidence:.2f}"
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 3)
            cv2.putText(image,label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis("off")
plt.show()